#  Exercise 3

## Installing ffmpeg and ffprobe
- ffmpeg and ffprrobe were already installed before implementing exercise 3.
- if they are not installed, then they have to be installed by utilities (curl, tar)
 -  below code are from exercise19 (week19)

In [1]:
# Download latest FFmpeg static build.  
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
else:
    print('ffmpeg: already installed')
!which ffmpeg

ffmpeg: already installed
/usr/bin/ffmpeg


## Importing modules
- os.path module is needed

In [2]:
import os.path

## Setting DEBUG_MODE variable
-  DEBUG_MODE variable is used to control verbosity
 - if this variable is set as True then the application print verbosly
 - else the application print silently

In [3]:
DEBUG_MODE = False

## Function Definition: get_Movie_information
-  this function returns digital fields of an input movie file
 - input: path of a movie file
 - output: list of the digital fields (e.g. video codec, audio codecc)

In [4]:
def get_Movie_Information(movie_path):
    ret_value = []    
    video_container = movie_path[movie_path.rfind('.') + 1:] 
    video_codec     = !ffprobe -v error -select_streams v -show_entries stream=codec_name -of csv=p=0:s=x  {movie_path}
    audio_codec     = !ffprobe -v error -select_streams a -show_entries stream=codec_name -of csv=p=0:s=x  {movie_path}
    frame_rate      = !ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate {movie_path}
    aspect_ratio    = !ffprobe -v error -select_streams v:0 -show_entries stream=display_aspect_ratio -of default=noprint_wrappers=1:nokey=1 {movie_path}
    resolution      = !ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of default=nw=1 {movie_path}
    video_bit_rate  = !ffprobe -v quiet -select_streams v:0 -show_entries stream=bit_rate -of default=noprint_wrappers=1 {movie_path}  
    audio_bit_rate  = !ffprobe -v quiet -select_streams a:0 -show_entries stream=bit_rate -of default=noprint_wrappers=1 {movie_path}  
    audio_channels  = !ffprobe -show_entries stream=channels -select_streams a:0 -of compact=p=0:nk=1 -v 0  -i  {movie_path}
    ret_value.append(video_container)
    ret_value.append(video_codec[0])
    ret_value.append(audio_codec[0])
    if DEBUG_MODE:
        print('frame_rate->' + str(frame_rate))
    ret_value.append(eval(frame_rate[0]))
    resolution_to_aspect_ratio = {
        '628x354': '314:177',
        '640x360': '16:9',
        '720x404': '180:101'
    }
    str_resolution = resolution[0].split('=')[1] + 'x' +resolution[1].split('=')[1]
    aspect_ratio = resolution_to_aspect_ratio[ str_resolution ] 
    ret_value.append(aspect_ratio)
    ret_value.append(str_resolution)
    ret_value.append(eval(video_bit_rate[0][9:])/pow(10, 6))
    ret_value.append(eval(audio_bit_rate[0][9:])/pow(10, 3))
    ret_value.append('stereo' if audio_channels[0] == '2' else 'mono')
    return ret_value

## Function Definition: get_required_Movie_Information
-  this function returns the required digital fields of the narbonne film festival
 - no input
 - output: the list of the required digital fields (e.g. video codec, audio codecc)

In [5]:
def get_required_Movie_Information():
    ret_value = []
    required_video_format = 'mp4'
    required_video_codec = 'h264'
    required_audio_codec = 'aac'
    required_frame_rate = 25.0
    required_aspect_ratio = '16:9'  
    required_resolution = '640x360'
    required_min_video_bit_rate_Mbs = 2
    required_max_video_bit_rate_Mbs = 5    
    required_max_audio_bit_rate_Kbs = 256
    required_audio_channels = 'stereo'   
    
    ret_value.append(required_video_format)
    ret_value.append(required_video_codec)
    ret_value.append(required_audio_codec)
    ret_value.append(required_frame_rate)
    ret_value.append(required_aspect_ratio)
    ret_value.append(required_resolution)
    ret_value.append([required_min_video_bit_rate_Mbs, required_max_video_bit_rate_Mbs])
    ret_value.append([0, required_max_audio_bit_rate_Kbs])
    ret_value.append(required_audio_channels)
    return ret_value

## Function Definition: compare_Movie_Information
-  this function returns the result of comparison of two movie information
 - input: two lists of movie information. usually one is a submitted films' fields, the other is the required fields
 - output: the list of the result of comparing (e.g. if their video formats are same, then the first list element is True )

In [6]:

def compare_Movie_Information(info_movie, info_required):
    index_video_format = 0
    index_video_codec = 1
    index_audio_codec = 2
    index_frame_rate = 3
    index_aspect_ratio = 4
    index_resolution = 5
    index_video_bit_rate = 6
    index_audio_bit_rate = 7
    index_audio_channels = 8
    ret_value = []
    ret_value.append(True if info_movie[index_video_format] == info_required[index_video_format] else False)
    ret_value.append(True if info_movie[index_video_codec] == info_required[index_video_codec] else False)
    ret_value.append(True if info_movie[index_audio_codec] == info_required[index_audio_codec] else False)
    ret_value.append(True if info_movie[index_frame_rate] == info_required[index_frame_rate] else False)
    ret_value.append(True if info_movie[index_aspect_ratio] == info_required[index_aspect_ratio] else False)
    ret_value.append(True if info_movie[index_resolution] == info_required[index_resolution] else False)
    ret_value.append(True if info_required[index_video_bit_rate][0] < info_movie[index_video_bit_rate] < info_required[index_video_bit_rate][1] else False)
    ret_value.append(True if info_required[index_audio_bit_rate][0] < info_movie[index_audio_bit_rate] < info_required[index_audio_bit_rate][1] else False)
    ret_value.append(True if info_movie[index_audio_channels] == info_required[index_audio_channels] else False)
    return ret_value

## Function Definition: get_converted_Movie
-  this function converts input movie as required
 - input: the path of the submitted movie file
 - output: the path of the converted movie file (e.g. osmos_War_of_the_Planets_formatOK.mp4)

In [7]:
def get_converted_Movie(movie_path):
    info_required = get_required_Movie_Information()
    info_movie = get_Movie_Information(movie_path)
    info_compared = compare_Movie_Information(info_movie, info_required)

    index_frame_rate, index_resolution, index_video_bit_rate, index_audio_bit_rate = [3, 5, 6, 7]
    
    same_resolution = True if info_compared[index_resolution] else False
    
    if not same_resolution:
        output_path_intermediate = movie_path[:movie_path.rfind('.')] + ('_im' if info_compared[0] else '') + '.mp4'       
        res = !ffmpeg -hide_banner -loglevel error -y -i {movie_path} -vf scale=640:360 {output_path_intermediate}
                
    info_movie = get_Movie_Information(movie_path if same_resolution else output_path_intermediate)            
    info_compared = compare_Movie_Information(info_movie, info_required)    
#    output_path = movie_path[:movie_path.rfind('.')] + ('_formatOK' if info_compared[0] else '') + '.mp4'   
    output_path = movie_path[:movie_path.rfind('.')] + '_formatOK' + '.mp4'   
    
    fps_parameter = '' if info_compared[index_frame_rate] else '-filter:v fps=25'
    video_bit_rate_parameter = '' if info_compared[index_video_bit_rate] else '-b:v 3M'
    audio_bit_rate_parameter = '' if info_compared[index_audio_bit_rate] else '-b:a 256k'
    
    input_path = movie_path if same_resolution else output_path_intermediate
    if DEBUG_MODE:
        print(f'output_path: {output_path}')
    res = !ffmpeg -hide_banner  -loglevel error  -y  -i {input_path} {fps_parameter}  {video_bit_rate_parameter} {audio_bit_rate_parameter} {output_path}

    if not same_resolution:
        !rm {output_path_intermediate}

    return output_path

## Function Definition: print_compared_Info
-  this function returns the formatted result of comparison of two movies
 - input: two lists of two movies, the (unformatted) result comparison of two movies  
 - output: the formatted result of comparison (string) 

In [8]:
def print_compared_Info(info_movie, info_required, info_compared):    
    str_items = [
        'video_format', 
        'video_codec', 
        'audio_codec', 
        'frame_rate', 
        'aspect_ratio', 
        'resolution', 
        'video_bit_rate(Mb/s)', 
        'audio_bit_rate(Kb/s)',
        'audio_channels'
    ]
    ret_value = ""
    for item, movie, required, compared in zip(str_items, info_movie, info_required, info_compared):
        str_value = f'{item}: {movie} {"=> satisfied" if compared else "===> not satisfied"}' 
        ret_value = ret_value + str_value + '\n'
    ret_value = ret_value + '-' * 20 + '\n'
    return ret_value

## Function Definition: generate_Report_Text_File
-  this function generates the formatted result of comparison of two movies
 - input: the file name of report, the list of files to be analyzed     
 - no return value 

In [9]:
def generate_Report_Text_File(REPORT_FILE_NAME, original_File_Paths):
    info_required = get_required_Movie_Information()
    str_value = ""   
    for original_File_Path in original_File_Paths:
        info_movie = get_Movie_Information(original_File_Path)
        str_value = str_value + 'File Name: '+ original_File_Path + '\n'
        info_compared = compare_Movie_Information(info_movie, info_required)
        str_value = str_value + print_compared_Info(info_movie, info_required, info_compared)

    with open(REPORT_FILE_NAME, 'w') as file:
        file.write(str_value)   

## The Exercise3 Requirement 0: Generating the Report File 
-  the report file name is 'report.txt'
 - this report file is generated by calling generate_Report_Text_File function
 - Cosmos_War_of_the_Planets.mp4: frame rate, aspect ratio, resolution, audio bit rate ==> not satisfied
 - The_Hill_Gang_Rides_Again.mp4: video bit rate, audio bit rate ==> not satisfied
 - Voyage_to_the_Planet_of_Prehistoric_Women.mp4: video codec, audio codec, frame rate, video bit rate, audio bit rate ==> not satisfied
 - Last_man_on_earth_1964.mov: video format, video codec, audio codec, frame rate, video bit rate, audio bit rate ==> not satisfied 
 - The_Gun_and_the_Pulpit.avi: video format, video codec, audio codec, aspect ratio, resolution, video bit rate, audio bit rate ==> not satisfied

In [10]:
REPORT_FILE_NAME = 'report.txt'
ORIGINAL_FILE_PATHS = [
    './original_videos/Cosmos_War_of_the_Planets.mp4',
    './original_videos/The_Hill_Gang_Rides_Again.mp4',
    './original_videos/Voyage_to_the_Planet_of_Prehistoric_Women.mp4',
    './original_videos/Last_man_on_earth_1964.mov',
    './original_videos/The_Gun_and_the_Pulpit.avi'    
]    
generate_Report_Text_File(REPORT_FILE_NAME, ORIGINAL_FILE_PATHS)
! cat {REPORT_FILE_NAME}

File Name: ./original_videos/Cosmos_War_of_the_Planets.mp4
video_format: mp4 => satisfied
video_codec: h264 => satisfied
audio_codec: aac => satisfied
frame_rate: 29.97002997002997 ===> not satisfied
aspect_ratio: 314:177 ===> not satisfied
resolution: 628x354 ===> not satisfied
video_bit_rate(Mb/s): 2.989377 => satisfied
audio_bit_rate(Kb/s): 317.103 ===> not satisfied
audio_channels: stereo => satisfied
--------------------
File Name: ./original_videos/The_Hill_Gang_Rides_Again.mp4
video_format: mp4 => satisfied
video_codec: h264 => satisfied
audio_codec: aac => satisfied
frame_rate: 25.0 => satisfied
aspect_ratio: 16:9 => satisfied
resolution: 640x360 => satisfied
video_bit_rate(Mb/s): 7.53773 ===> not satisfied
audio_bit_rate(Kb/s): 253.272 => satisfied
audio_channels: stereo => satisfied
--------------------
File Name: ./original_videos/Voyage_to_the_Planet_of_Prehistoric_Women.mp4
video_format: mp4 => satisfied
video_codec: hevc ===> not satisfied
audio_codec: mp3 ===> not satisf

## Function Definition: generate_Movie_File
-  this function generates the converted movie files as required 
 - input: the list of files to be converted 
 - no return value 

In [11]:
def generate_Movie_File(original_File_Paths):
    info_required = get_required_Movie_Information()
    #str_value = ""   
    for original_File_Path in original_File_Paths:
        info_movie = get_Movie_Information(original_File_Path)

        info_compared = compare_Movie_Information(info_movie, info_required)
        if not all(info_compared):
            encoded_File_Path = get_converted_Movie(original_File_Path)     
            info_movie = get_Movie_Information(encoded_File_Path)
            info_compared = compare_Movie_Information(info_movie, info_required)        
            print(f'{original_File_Path} -> {encoded_File_Path}')
            
            if all(info_compared):
                print('now all True')
            else:
                print('still not all True')  

## The Exercise3 Requirement 1: Generating the converted Movie Files 
-  generating new movie files which are converted as required

In [12]:
generate_Movie_File(ORIGINAL_FILE_PATHS)

./original_videos/Cosmos_War_of_the_Planets.mp4 -> ./original_videos/Cosmos_War_of_the_Planets_formatOK.mp4
now all True
./original_videos/The_Hill_Gang_Rides_Again.mp4 -> ./original_videos/The_Hill_Gang_Rides_Again_formatOK.mp4
now all True
./original_videos/Voyage_to_the_Planet_of_Prehistoric_Women.mp4 -> ./original_videos/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
now all True
./original_videos/Last_man_on_earth_1964.mov -> ./original_videos/Last_man_on_earth_1964_formatOK.mp4
now all True
./original_videos/The_Gun_and_the_Pulpit.avi -> ./original_videos/The_Gun_and_the_Pulpit_formatOK.mp4
now all True


## Confirming the result of converting
- confirming the result of converting by generating the report of converted files
- the new report file name is 'new_report.txt'
- this result: all files are converted as required
 - Cosmos_War_of_the_Planets_formatOK.mp4: all satisfied
 - The_Hill_Gang_Rides_Again_formatOK.mp4: all satisfied
 - Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4: all satisfied
 - Last_man_on_earth_1964_formatOK.mp4: all satisfied
 - The_Gun_and_the_Pulpit_formatOK.mp4: all satisfied

In [13]:
NEW_REPORT_FILE_NAME = 'new_report.txt'
NEW_FILE_PATHS = [
    './original_videos/Cosmos_War_of_the_Planets_formatOK.mp4',
    './original_videos/The_Hill_Gang_Rides_Again_formatOK.mp4',
    './original_videos/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4',
    './original_videos/Last_man_on_earth_1964_formatOK.mp4',
    './original_videos/The_Gun_and_the_Pulpit_formatOK.mp4'    
]    
generate_Report_Text_File(NEW_REPORT_FILE_NAME, NEW_FILE_PATHS)
! cat {NEW_REPORT_FILE_NAME}

File Name: ./original_videos/Cosmos_War_of_the_Planets_formatOK.mp4
video_format: mp4 => satisfied
video_codec: h264 => satisfied
audio_codec: aac => satisfied
frame_rate: 25.0 => satisfied
aspect_ratio: 16:9 => satisfied
resolution: 640x360 => satisfied
video_bit_rate(Mb/s): 2.971616 => satisfied
audio_bit_rate(Kb/s): 129.606 => satisfied
audio_channels: stereo => satisfied
--------------------
File Name: ./original_videos/The_Hill_Gang_Rides_Again_formatOK.mp4
video_format: mp4 => satisfied
video_codec: h264 => satisfied
audio_codec: aac => satisfied
frame_rate: 25.0 => satisfied
aspect_ratio: 16:9 => satisfied
resolution: 640x360 => satisfied
video_bit_rate(Mb/s): 2.982541 => satisfied
audio_bit_rate(Kb/s): 128.962 => satisfied
audio_channels: stereo => satisfied
--------------------
File Name: ./original_videos/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
video_format: mp4 => satisfied
video_codec: h264 => satisfied
audio_codec: aac => satisfied
frame_rate: 25.0 => satisf